# Variational Inference

### Estimation of Transition matrix and Emission parameters for each state

Libraries:

In [4]:
import numpy as np
import torch
import pyro
from pyro import poutine
import pyro.distributions as dist
from pyro.infer import SVI, TraceEnum_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam
import pandas as pd
import tqdm
from models.UnivariateHMM import UnivariateHMM
from models.CopulaHMM import CopulaHMM
from utils.CopulaHelpers import *

Global variables:

In [2]:
DATA_DIR="data/"
HIDDEN_STATES = 2
TRAINING_STEPS= 500

Import data:

In [3]:
data = pd.read_csv(f"{DATA_DIR}hulls_df_matchday2_reduced.csv")
data = data.dropna()
# Convert the areas from m^2 to dam^2 for computational reasons
data["HomeHull"]=data["HomeHull"]/100
data["AwayHull"]=data["AwayHull"]/100
data.head()

,Time [s],Period,HomeHull,AwayHull
0,2.0,1.0,5.982854,8.106641
1,4.0,1.0,6.470239,9.445797
2,6.0,1.0,6.687160,10.615702
3,8.0,1.0,6.578269,12.365166
4,10.0,1.0,6.843430,14.701596


## Univariate (separate) models

In [ ]:
sequence_X = torch.tensor(data["HomeHull"].values)
sequence_Y = torch.tensor(data["AwayHull"].values)

#### Home Team

In [ ]:
pyro.clear_param_store()

In [ ]:
# Guide
guide = AutoDelta(poutine.block(UnivariateHMM, expose=["probs_x", "probs_alpha", "probs_beta"]))
# Optimizer
optimizer = Adam({"lr": 0.01})
# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(UnivariateHMM, guide, optimizer, loss=elbo)
# Training
tqdm_bar = tqdm.tqdm(range(TRAINING_STEPS))
for step in tqdm_bar:
    loss = svi.step(sequence_X, HIDDEN_STATES)
    if step % 100 == 0:
         tqdm_bar.set_postfix({'LOSS': loss})

In [ ]:
posterior_HomeTeam = guide(sequence_X,HIDDEN_STATES)
posterior_HomeTeam

In [ ]:
torch.save(posterior_HomeTeam,f"parameters/univariateHMM_matchday2_HomeTeam_{HIDDEN_STATES}states.pt")

#### Away Team

In [ ]:
pyro.clear_param_store()

In [ ]:
# Guide
guide = AutoDelta(poutine.block(UnivariateHMM, expose=["probs_x", "probs_alpha", "probs_beta"]))
# Optimizer
optimizer = Adam({"lr": 0.01})
# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(UnivariateHMM, guide, optimizer, loss=elbo)
# Training
tqdm_bar = tqdm.tqdm(range(TRAINING_STEPS))
for step in tqdm_bar:
    loss = svi.step(sequence_Y, HIDDEN_STATES)
    if step % 100 == 0:
         tqdm_bar.set_postfix({'LOSS': loss})

In [ ]:
posterior_HomeTeam = guide(sequence_Y,HIDDEN_STATES)
posterior_HomeTeam

In [ ]:
torch.save(posterior_HomeTeam,f"parameters/univariateHMM_matchday2_AwayTeam_{HIDDEN_STATES}states.pt")

## Bivariate copula model

In [ ]:
sequence_XY = torch.tensor(data[["HomeHull","AwayHull"]].values)
sequence_XY.shape

In [ ]:
pyro.clear_param_store()

In [ ]:
# Optimizer
optimizer = Adam({"lr": 0.01})

# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(CopulaHMM, guide, optimizer, loss=elbo)

# Training
tqdm_bar = tqdm.tqdm(range(TRAINING_STEPS))
for step in tqdm_bar:
    loss = svi.step(sequence_XY, HIDDEN_STATES)
    #if step % 50 == 0:
    tqdm_bar.set_postfix({'LOSS': loss})

In [ ]:
posterior = guide(sequence_XY,HIDDEN_STATES)
posterior

In [ ]:
torch.save(posterior,f"parameters/CopulaHMM_matchday2_{HIDDEN_STATES}states.pt")